In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from math import sqrt
import scipy.stats as st

# Neural Network 
from numpy import array
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.optimizers import Adam # Custom optimizer


from data_preprocess import pre_processing # Used to preprocess data
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import class_weight

from data_preprocess import pre_processing # Used to preprocess data
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [ ]:
class NeuralNetworkAccel:
    def __init__(self, df, y_label, sequence_matters=False, 
                 random_state=42, train_size=0.8,
                 epochs=100, loss='binary_crossentropy', opt=None, 
                 validation_split=0.3, drop_out=0.25, 
                 num_nodes=64, num_hidden_layers=3,
                 learning_rate=0.0001, metrics = [keras.metrics.AUC()]
                 activation='relu', output_activation='sigmoid',
                 kernel_initializer='he_normal', class_weights=None,
                 **kwargs):
        """ 
        params:
        df: dataframe which we want to detect the anomaly
        y_label: the y label of which we are predicting

        """   
        df.default_ind = df.default_ind.apply(lambda x: int(not x)) # Flip 1 and 0
        y = df['default_ind']
        X = df.drop('default_ind', axis=1)
        
        # Train test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-train_size, shuffle=(not sequence_matters), random_state=random_state)
        X_train = X_train.copy() # Prevent warnings
        X_test = X_test.copy() # Prevent warnings
    
        # Scale data
        scalers = dict() # Save them in dictionary so we can rescale them back
        for col in X_train.columns:
            if len(X_train[col].unique()) != 2:
                scalers[col] = StandardScaler()
                scalers[col].fit(X_train[col].values.reshape(-1, 1))
                X_train[col] = scalers[col].transform(X_train[col].values.reshape(-1, 1))
                X_test[col] = scalers[col].transform(X_test[col].values.reshape(-1, 1))
                
        self.scalers = scalers
        
        # Dense Neural Network Model
        num_features = len(X_train.columns)

        model = Sequential()
        model.add(Dense(num_nodes, input_dim=num_features, activation=activation))
        for _ in range(num_hidden_layers - 1):
            model.add(Dense(num_nodes, activation=activation, kernel_initializer = "he_normal"))
            model.add(Dropout(drop_out))
        model.add(Dense(num_nodes, activation=activation, kernel_initializer = "he_normal"))
        model.add(Dense(1, activation=output_activation))
        
        # Create weights for unbalanced data
        if class_weights is None:
            class_weights = class_weight.compute_class_weight('balanced',
                                                             np.unique(y_train),
                                                             y_train)
        if opt is None:
            opt = Adam(lr=learning_rate)
        
        model.compile(loss=loss, optimizer=opt, metrics=metrics)

        history = model.fit(X_train.values, y_train, epochs=epochs, validation_split=validation_split, class_weight=class_weights)
        
        self.model = model
        
    def predict(self, data):
        """
        This method predicts anomalies given a dataset that this class was trained on,
        the dataset must have the same columns as the trained model, it will output the 
        an array of anomalies, also returns probabilities for each for ROC_AUC scores
        """
        return self.model.predict(data), self.model.predict_proba(data)[:, 1]
    
        @staticmethod
    def get_scores(y_pred, y_true, scoring='f1', y_proba=None):
        if scoring.startswith('f1'):
            score = f1_score(y_pred, y_true)
        elif scoring.startswith('pre'):
            score = precision_score(y_pred, y_true)
        elif scoring.startswith('rec'):
            score = recall_score(y_pred, y_true)
        elif scoring.startswith('acc'):
            score = accuracy_score(y_pred, y_true)
        elif scoring.startswith('roc'):
            if  y_proba is None:
                score = 'ROC AUC requires probability scores'
            else:
                score = roc_auc_score(y_pred, y_proba)
        else:
            score = f"Scoring Method: {scoring} not implemented"
        return score
    
    @staticmethod
    def get_all_scores(y_pred, y_true, y_proba):
        # This just prints all scores above above
        print(f"ROC_AUC: {LightGBMAccel.get_scores(y_pred, y_true, scoring='roc', y_proba=y_proba):0.2f}")
        print(f"F1: {LightGBMAccel.get_scores(y_pred, y_true, scoring='f1'):0.2f}")
        print(f"Precision: {LightGBMAccel.get_scores(y_pred, y_true, scoring='pre'):0.2f}")
        print(f"Recall: {LightGBMAccel.get_scores(y_pred, y_true, scoring='rec'):0.2f}")
        print(f"Accuracy: {LightGBMAccel.get_scores(y_pred, y_true, scoring='acc'):0.2f}")

    @staticmethod
    def get_confusion_matrix(y_pred, y_true, cmap='coolwarm'):
        plt.figure(figsize=(10,8))
        
        cf_matrix = confusion_matrix(y_test, y_true)
        group_names = ['Non-Anomalies', 'Falsely Flagged Anom', 'Unflagged Anom', 'True Anomalies']
        group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
        group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
        
        # Labels combine group names, counts, and percentages
        labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names, group_counts, group_percentages)] 
        labels = np.asarray(labels).reshape(2,2)
              
        sns.heatmap(cf_matrix, annot=labels, fmt="", cmap=cmap)

In [2]:
df = pre_processing(pd.read_csv("data.csv"))

In [3]:
df.default_ind = df.default_ind.apply(lambda x: int(not x)) # Flip 1 and 0

In [4]:
y = df['default_ind']
X = df.drop('default_ind', axis=1)

In [6]:
# Additional preprocessing for neural networks

# Scale data
scalers = dict() # Save them in dictionary so we can rescale them back
for col in X_train.columns:
    if len(X_train[col].unique()) != 2:
        scalers[col] = StandardScaler()
        scalers[col].fit(X_train[col].values.reshape(-1, 1))
        X_train[col] = scalers[col].transform(X_train[col].values.reshape(-1, 1))
        X_test[col] = scalers[col].transform(X_test[col].values.reshape(-1, 1))

In [49]:
num_features = len(X_train.columns)

model = Sequential()
model.add(Dense(128, input_dim=num_features, activation='relu'))
model.add(Dense(128, activation='relu', kernel_initializer = "he_normal"))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu', kernel_initializer = "he_normal"))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu', kernel_initializer = "he_normal"))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu', kernel_initializer = "he_normal"))
model.add(Dense(1, activation='sigmoid'))

from sklearn.utils import class_weight
from tensorflow.keras.optimizers import Adam
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
opt = Adam(lr=0.0001)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC()])

history = model.fit(X_train.values, y_train, epochs=150, validation_split=0.3, class_weight=class_weights)

Train on 4900 samples, validate on 2100 samples
Epoch 1/150
4900/4900 [==============================] - 1s 102us/step - loss: 0.4758 - auc_7: 0.4901 - val_loss: 0.4397 - val_auc_7: 0.5715
Epoch 2/150
4900/4900 [==============================] - 0s 65us/step - loss: 0.4479 - auc_7: 0.5390 - val_loss: 0.4365 - val_auc_7: 0.6129
Epoch 3/150
4900/4900 [==============================] - 0s 64us/step - loss: 0.4391 - auc_7: 0.5525 - val_loss: 0.4299 - val_auc_7: 0.6387
Epoch 4/150
4900/4900 [==============================] - 0s 64us/step - loss: 0.4252 - auc_7: 0.6047 - val_loss: 0.4190 - val_auc_7: 0.6508
Epoch 5/150
4900/4900 [==============================] - 0s 65us/step - loss: 0.4250 - auc_7: 0.6063 - val_loss: 0.4217 - val_auc_7: 0.6597
Epoch 6/150
4900/4900 [==============================] - 0s 64us/step - loss: 0.4202 - auc_7: 0.6266 - val_loss: 0.4214 - val_auc_7: 0.6652
Epoch 7/150
4900/4900 [==============================] - 0s 63us/step - loss: 0.4157 - auc_7: 0.6436 - val_loss

Epoch 59/150
4900/4900 [==============================] - 0s 68us/step - loss: 0.2547 - auc_7: 0.9110 - val_loss: 0.5514 - val_auc_7: 0.6373
Epoch 60/150
4900/4900 [==============================] - 0s 67us/step - loss: 0.2529 - auc_7: 0.9130 - val_loss: 0.5505 - val_auc_7: 0.6391
Epoch 61/150
4900/4900 [==============================] - 0s 65us/step - loss: 0.2513 - auc_7: 0.9130 - val_loss: 0.5535 - val_auc_7: 0.6314
Epoch 62/150
4900/4900 [==============================] - 0s 64us/step - loss: 0.2394 - auc_7: 0.9235 - val_loss: 0.5728 - val_auc_7: 0.6309
Epoch 63/150
4900/4900 [==============================] - 0s 64us/step - loss: 0.2344 - auc_7: 0.9275 - val_loss: 0.5911 - val_auc_7: 0.6336
Epoch 64/150
4900/4900 [==============================] - 0s 67us/step - loss: 0.2304 - auc_7: 0.9294 - val_loss: 0.5863 - val_auc_7: 0.6306
Epoch 65/150
4900/4900 [==============================] - 0s 65us/step - loss: 0.2289 - auc_7: 0.9307 - val_loss: 0.5976 - val_auc_7: 0.6296
Epoch 66/150


Epoch 117/150
4900/4900 [==============================] - 0s 68us/step - loss: 0.0684 - auc_7: 0.9947 - val_loss: 1.3925 - val_auc_7: 0.5757
Epoch 118/150
4900/4900 [==============================] - 0s 66us/step - loss: 0.0739 - auc_7: 0.9933 - val_loss: 1.4088 - val_auc_7: 0.5776
Epoch 119/150
4900/4900 [==============================] - 0s 62us/step - loss: 0.0699 - auc_7: 0.9932 - val_loss: 1.3788 - val_auc_7: 0.5871
Epoch 120/150
4900/4900 [==============================] - 0s 66us/step - loss: 0.0742 - auc_7: 0.9935 - val_loss: 1.4072 - val_auc_7: 0.5820
Epoch 121/150
4900/4900 [==============================] - 0s 64us/step - loss: 0.0724 - auc_7: 0.9938 - val_loss: 1.4388 - val_auc_7: 0.5765
Epoch 122/150
4900/4900 [==============================] - 0s 60us/step - loss: 0.0758 - auc_7: 0.9922 - val_loss: 1.3982 - val_auc_7: 0.5827
Epoch 123/150
4900/4900 [==============================] - 0s 67us/step - loss: 0.0588 - auc_7: 0.9954 - val_loss: 1.4303 - val_auc_7: 0.5797
Epoch 

In [46]:
pred = model.predict(X_test)

In [47]:
y_pred = [0 if x < 0.5 else 1 for x in pred ]

In [50]:
pred

array([[0.9867319 ],
       [0.9999631 ],
       [0.7871119 ],
       ...,
       [0.9999994 ],
       [0.99998283],
       [1.        ]], dtype=float32)

In [48]:
confusion_matrix(y_test, y_pred)

array([[  79,  369],
       [ 225, 2327]], dtype=int64)